In [1]:
import os
import sys
sys.path.append(os.getcwd()+"/../..")
from src import paths

import pandas as pd
import numpy as np

import torch

from sklearn.metrics.pairwise import cosine_similarity

from sklearn.metrics import f1_score, recall_score, precision_score, classification_report

import warnings
warnings.filterwarnings("ignore")

In [8]:
pd.set_option("max_colwidth", None)

In [5]:
def load_results(filename:str, labels:str, load_line:bool = False, load_outlines:bool = False, label_correction:dict = None):
    """ 
    Load results from a file and return a pandas dataframe with the results.

    Args:
        filename (str): The name of the file to load.
        labels (str): The name of the file containing the labels.
        load_line (bool, optional): If True, the function will expect a line labelled dataset and return the aggregated results per rid.
        load_outlines (bool, optional): If True, the function will expect the output of an outlines prompt (no hidden states and model answers are labels)
        label_correcttion (dict, optional): A dictionary with rids as keys and the corrected labels as values.

    Returns:
        pd.DataFrame: A pandas dataframe with the results.

    """
    results = torch.load(paths.RESULTS_PATH/"ms-diag"/filename, map_location=torch.device('cpu'))
    labels = torch.load(paths.RESULTS_PATH/"ms-diag"/labels, map_location=torch.device('cpu'))

    output = {"model_answers": results.pop("model_answers")}

    if not load_outlines:
        
        # Get prediction through cosine similarity
        last_hidden_states = results.pop("last_hidden_states")
        last_hidden_states = last_hidden_states.cpu()
        preds = get_prediction(last_hidden_states, labels_hs=labels[1], label_names=labels[0])

        # If rag
        if "rag" in filename:
            output["model_answers_mapped"] = preds + ["no information found" for _ in range(len(output["model_answers"])-len(preds))]
        else:
            output["model_answers_mapped"] = preds
    
    # Map from string answer to int
    label2id = {'primär progrediente Multiple Sklerose': 0,
            'sekundär progrediente Multiple Sklerose': 2,
            'schubförmige remittierende Multiple Sklerose': 1,
            'other': 3,
            'no information found': 3}
    
    if load_outlines:
        key = "model_answers"
    else:
        key = "model_answers_mapped"
    
    output["preds"] = [label2id[i] for i in output[key]]
    output["exact_match"] = [res.lower().split("### output:\n")[-1] in [key.lower() for key in label2id.keys()] for res in output["model_answers"]]

    output["labels"] = results["labels"]
    output["rid"] = results["rid"]
    output["text"] = results["text"]

    if "whole_prompt" in results.keys():
        whole_prompt = results["whole_prompt"]
        if "rag" in filename:
            output["whole_prompt"] = whole_prompt + ["no information found" for _ in range(len(output["model_answers"])-len(whole_prompt))]
        else:
            output["whole_prompt"] = whole_prompt
    
    df = pd.DataFrame(output)

    # Correcting wrong labels from analysis:
    if label_correction:
        for rid, label in label_correction.items():
            df.loc[df.rid == rid, "labels"] = label
    

    if load_line:
        return convert_line2report(df, filename)
    else:
        return df

def get_prediction(hs: torch.Tensor, labels_hs: torch.Tensor, label_names: list):
    """ 
    Get the prediction for a given hidden state and labels.

    Args:
        hs (torch.Tensor): The hidden state to use for prediction. Shape (n_samples, n_features).
        labels_hs (torch.Tensor): The hidden states of the labels. Shape (n_labels, n_features).
        label_names (list): The names of the labels. The order of the names should match the order of the hs-labels in dim 0.

    Returns:
        torch.Tensor: The predicted labels.

    """
    answer_idx = np.argmax(cosine_similarity(hs, labels_hs), axis=1)
    model_answer = [label_names[i] for i in answer_idx]
    return model_answer

def convert_line2report(rids:list[str], preds:list[int])->list[int]:
    """ 
    Aggregeates the results of a line labelled dataset in a majority vote fashion.

    Args:
        rids (list[str]): The rids of the samples.
        preds (list[int]): The predictions of the samples.

    Returns:
        list[int]: The aggregated predictions.
        
    """
    df = pd.DataFrame({"rid":rids, "preds":preds})

    # If len value counts >1 and value counts[0] is 3, then majority vote is value counts[1]
    results = []
    rid = []
    for _df in df.groupby("rid"):
        if _df[1].preds.value_counts().shape[0] > 1 and _df[1].preds.value_counts().iloc[0] == 3:
            result = df.loc[df.rid == _df[0], "preds"] = _df[1].preds.value_counts().index[1]
        else:
            result = df.loc[df.rid == _df[0], "preds"] = _df[1].preds.value_counts().index[0]
        results.append(result)
        rid.append(_df[0])
    return result


def summarize_performance(files: list[str], *args, **kwargs):

    """ 
    Summarizes the performance of a given strategy (Base, RAG, Outlines) for all prompting strategies.

    Args:
        files (list[str]): The files to summarize.
        *args: Variable length argument list.
        **kwargs: Arbitrary keyword arguments.

    Returns:
        pd.DataFrame: A pandas dataframe with the summarized results.

    """
    dfs = []
    for filename in files:
        strategies = ["zero_shot_vanilla", "zero_shot_instruction", "few_shot_vanilla", "few_shot_instruction"]
        
        # Set strategy name to whatever is found in the filename
        for strategy in strategies:
            if strategy in filename:
                break

        results = load_results(filename=filename, labels=kwargs.get("labels"), load_line=kwargs.get("load_lines"), load_outlines=kwargs.get("load_outlines"), label_correction=kwargs.get("label_correction"))

        if kwargs.get("verbose"):
            display(results[results["labels"] != results["preds"]])

        # Target names
        metric_dict = classification_report(y_true=results["labels"], y_pred=results["preds"], output_dict=True)

        # Create a dictionary with flattened keys
        _df = pd.json_normalize(metric_dict)
        

        # Add additional information
        _df["strategy"] = strategy

        # Look for exact label
        _df["valid_label"] = sum(results["exact_match"])/len(results)

        # Reorder columns
        reordered_cols = _df.columns[-4:].append(_df.columns[:-4])
        _df = _df[reordered_cols]

        dfs.append(_df)

    return pd.concat(dfs)

def generate_latex_table(df: pd.DataFrame, caption: str, label: str, metrics = list[str]) -> str:
    """
    Generate LaTeX code for a table from a pandas DataFrame.

    Args:
        df (pd.DataFrame): DataFrame containing the table data.
        caption (str): Caption for the table.
        label (str): Label for referencing the table.
        metrics (list[str]): List of metrics to include in the table.

    Returns:
        str: LaTeX code for the table.
    """
    # Create the LaTeX code for the table
    latex = "\\begin{table}[h]\n"
    latex += "    \\centering\n"

    # Create tables for each metric
    for metric in metrics:
        latex += f"    \\begin{{tabular}}{{lcccc}}\n"
        latex += f"        \\toprule\n"
        latex += f"        & \\multicolumn{{4}}{{c}}{{{metric}}}\\\\\n"
        latex += f"        \\cmidrule(lr){{2-5}}\n"
        latex += f"        & Base & +S2A & +Outlines & +S2A \\& Outlines \\\\\n"
        latex += f"        \\midrule\n"

        for strategy in df['strategy'].unique():
            base_value = df[(df['strategy'] == strategy) & (df['approach'] == 'base')][metric].iloc[0]
            rag_value = df[(df['strategy'] == strategy) & (df['approach'] == 'rag')][metric].iloc[0]
            outlines_value = df[(df['strategy'] == strategy) & (df['approach'] == 'outlines')][metric].iloc[0]
            outlines_rag_value = df[(df['strategy'] == strategy) & (df['approach'] == 'outlines_rag')][metric].iloc[0]
            
            latex += f"        {strategy.replace('_', '-')} & {base_value:.2f} & {rag_value:.2f} & {outlines_value:.2f} & {outlines_rag_value:.2f} \\\\\n"

        latex += f"        \\bottomrule\n"
        latex += f"    \\end{{tabular}}\n\n"

    # Add caption and label
    latex += f"    \\caption{{{caption}}}\n"
    latex += f"    \\label{{tab:{label}}}\n"
    latex += "\\end{table}\n"

    return latex

# Llama2-MedTuned-13B

## RAG

In [9]:
pd.set_option('display.max_columns', None)
summarize_performance(["ms-diag_Llama2-MedTuned-13b_4bit_all_test_zero_shot_vanilla_rag.pt",
                       "ms-diag_Llama2-MedTuned-13b_4bit_all_test_zero_shot_instruction_rag.pt",
                       "ms-diag_Llama2-MedTuned-13b_4bit_all_test_few_shot_vanilla_rag.pt",
                       "ms-diag_Llama2-MedTuned-13b_4bit_all_test_few_shot_instruction_rag.pt",
                       ], labels="label_encodings_Llama2-MedTuned-13b.pt", load_line=False, load_outlines=False, verbose = True)

,model_answers,model_answers_mapped,preds,exact_match,labels,rid,text,whole_prompt
9,schubförmig remittierende multiple sklerose (rrms),primär progrediente Multiple Sklerose,0,False,1,08F1FA27-6213-47AF-B967-DC44CCCE9613,"Schubförmig remittierende Multiple Sklerose (EM 09/14), EDSS 0.0","[INST]<<SYS>>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.\nYour answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.\nPlease ensure that your responses are socially unbiased and positive in nature.\nIf a question does not makeany sense, or is not factually coherent, explain why instead of answering something not correct. \nIf you don’t know the answer to a question, please don’t share false information.\n<</SYS>>\n\nYour task is to extract the diagnosis corresponding to a type of multiple sclerosis (MS) stated in a German medical report. The input for this task is a German medical report, and the output should be the type of MS.\nThere are three types of multiple sclerosis in German:\n- primär progrediente Multiple Sklerose (PPMS)\n- sekundär progrediente Multiple Sklerose (SPMS)\n- schubförmig remittierende Multiple Sklerose (RRMS)\n\nThe type is provided in the text, and your task is to extract it. If you cannot match a type exactly, please answer with 'other'.\nYour answer should solely consist of one of the following:\n- primär progrediente Multiple Sklerose\n- sekundär progrediente Multiple Sklerose\n- schubförmige remittierende Multiple Sklerose\n- other\n\nHere is the Input:\nSchubförmig remittierende Multiple Sklerose (EM 09/14), EDSS 0.0[/INST] Schubförmig remittierende Multiple Sklerose (RRMS)"
18,schubförmig remittierende multiple sklerose (rrms),primär progrediente Multiple Sklerose,0,False,1,73F77600-044F-49F4-8541-0077D90A2869,"Schubförmig remittierende Multiple Sklerose (RRMS), EDSS 2.0 (EM 1993, ED 2003)","[INST]<<SYS>>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.\nYour answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.\nPlease ensure that your responses are socially unbiased and positive in nature.\nIf a question does not makeany sense, or is not factually coherent, explain why instead of answering something not correct. \nIf you don’t know the answer to a question, please don’t share false information.\n<</SYS>>\n\nYour task is to extract the diagnosis corresponding to a type of multiple sclerosis (MS) stated in a German medical report. The input for this task is a German medical report, and the output should be the type of MS.\nThere are three types of multiple sclerosis in German:\n- primär progrediente Multiple Sklerose (PPMS)\n- sekundär progrediente Multiple Sklerose (SPMS)\n- schubförmig remittierende Multiple Sklerose (RRMS)\n\nThe type is provided in the text, and your task is to extract it. If you cannot match a type exactly, please answer with 'other'.\nYour answer should solely consist of one of the following:\n- primär progrediente Multiple Sklerose\n- sekundär progrediente Multiple Sklerose\n- schubförmige remittierende Multiple Sklerose\n- other\n\nHere is the Input:\nSchubförmig remittierende Multiple Sklerose (RRMS), EDSS 2.0 (EM 1993, ED 2003)[/INST] Schubförmig remittierende Multiple Sklerose (RRMS)"
19,other,other,3,True,1,96BC21AA-235F-4EED-A74F-58EFC11C1176,"Hochgradiger V.a. entzündliche ZNS-Erkrankung ED 03.05.2019, EM 30.04.2019\nINDENT ätiologisch: möglicherweise multiple Sklerose","[INST]<<SYS>>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.\nYour answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.\nPlease ensure that your responses are socially unbiased and positive in nature.\nIf a question does not makeany sense, or is not factually coherent, explain why

,model_answers,model_answers_mapped,preds,exact_match,labels,rid,text,whole_prompt
3,### output:\nschubförmig-remittierende multiple sklerose : rrms,primär progrediente Multiple Sklerose,0,False,1,69273DEA-ACAC-40CD-8F02-840EB3CC2027,"Schubförmig-remittierende Multiple Sklerose, EM und ED 04/2007\nINDENT EDSS 05/2019: 2.0","[INST]<<SYS>>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.\nYour answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.\nPlease ensure that your responses are socially unbiased and positive in nature.\nIf a question does not makeany sense, or is not factually coherent, explain why instead of answering something not correct. \nIf you don’t know the answer to a question, please don’t share false information.\n<</SYS>>\n### Instruction:\nYour task is to extract the diagnosis corresponding to a type of multiple sclerosis (MS) stated in a German medical report. The input for this task is a German medical report, and the output should be the type of MS.\nThere are three types of multiple sclerosis in German:\n- primär progrediente Multiple Sklerose (PPMS)\n- sekundär progrediente Multiple Sklerose (SPMS)\n- schubförmig remittierende Multiple Sklerose (RRMS)\n\nThe type is provided in the text, and your task is to extract it. If you cannot match a type exactly, please answer with 'other'.\nYour answer should solely consist of one of the following:\n- primär progrediente Multiple Sklerose\n- sekundär progrediente Multiple Sklerose\n- schubförmige remittierende Multiple Sklerose\n- other\n\n\n### Input:\nSchubförmig-remittierende Multiple Sklerose, EM und ED 04/2007\nINDENT EDSS 05/2019: 2.0[/INST]\n\n### Output:\nschubförmig-remittierende Multiple Sklerose : RRMS \n"
19,### output:\nother,other,3,True,1,96BC21AA-235F-4EED-A74F-58EFC11C1176,"Hochgradiger V.a. entzündliche ZNS-Erkrankung ED 03.05.2019, EM 30.04.2019\nINDENT ätiologisch: möglicherweise multiple Sklerose","[INST]<<SYS>>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.\nYour answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.\nPlease ensure that your responses are socially unbiased and positive in nature.\nIf a question does not makeany sense, or is not factually coherent, explain why instead of answering something not correct. \nIf you don’t know the answer to a question, please don’t share false information.\n<</SYS>>\n### Instruction:\nYour task is to extract the diagnosis corresponding to a type of multiple sclerosis (MS) stated in a German medical report. The input for this task is a German medical report, and the output should be the type of MS.\nThere are three types of multiple sclerosis in German:\n- primär progrediente Multiple Sklerose (PPMS)\n- sekundär progrediente Multiple Sklerose (SPMS)\n- schubförmig remittierende Multiple Sklerose (RRMS)\n\nThe type is provided in the text, and your task is to extract it. If you cannot match a type exactly, please answer with 'other'.\nYour answer should solely consist of one of the following:\n- primär progrediente Multiple Sklerose\n- sekundär progrediente Multiple Sklerose\n- schubförmige remittierende Multiple Sklerose\n- other\n\n\n### Input:\nHochgradiger V.a. entzündliche ZNS-Erkrankung ED 03.05.2019, EM 30.04.2019\nINDENT ätiologisch: möglicherweise multiple Sklerose[/INST]\n\n### Output:\nother \n"
25,### output:\nschubförmig remittierende multiple sklerose (rrms) :,primär progrediente Multiple Sklerose,0,False,1,1914ADB8-2E2D-4557-930E-4AB8422CBD5F,"Vd.a. autoimmun-entzündliche ZNS-Ekrankung, am ehesten einer schubförmig-remittierenden multiplen Sklerose entsprechend (EM 1997):","[INST]<<SYS>>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.\nYour answers should not include any harmful, unethical, racist, sexis

,model_answers,model_answers_mapped,preds,exact_match,labels,rid,text,whole_prompt
0,primär progrediente multiple sklerose,primär progrediente Multiple Sklerose,0,True,1,55E636B5-EA14-497D-832B-A87444F3C096,"Schubförmige Multiple Sklerose (EM, ED 12/2005), EDSS 1.0","[INST]<<SYS>>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.\nYour answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.\nPlease ensure that your responses are socially unbiased and positive in nature.\nIf a question does not makeany sense, or is not factually coherent, explain why instead of answering something not correct. \nIf you don’t know the answer to a question, please don’t share false information.\n<</SYS>>\n\nYour task is to extract the diagnosis corresponding to a type of multiple sclerosis (MS) stated in a German medical report. The input for this task is a German medical report, and the output should be the type of MS.\nThere are three types of multiple sclerosis in German:\n- primär progrediente Multiple Sklerose (PPMS)\n- sekundär progrediente Multiple Sklerose (SPMS)\n- schubförmig remittierende Multiple Sklerose (RRMS)\n\nThe type is provided in the text, and your task is to extract it. If you cannot match a type exactly, please answer with 'other'.\nYour answer should solely consist of one of the following:\n- primär progrediente Multiple Sklerose\n- sekundär progrediente Multiple Sklerose\n- schubförmige remittierende Multiple Sklerose\n- other\n\nHere are some examples to help you understand the task:\n\nInput:\nPrimär chronisch progrediente Multiple Sklerose, EM 2008, ED 2012, EDSS 4.0\naktuell: klinisch: stabil, radiologisch: stabil, Progression: ja (nach Lublin et al. 2013)\nINDENT Besserung der Kraft im lin...\nOutput:\nprimär progrediente Multiple Sklerose\n\nInput:\nSchubförmige Multiple Sklerose (EM, ED 12/2005), EDSS 1.0\nklinisch fraglich aktiv, radiologisch aktiv, keine Progression (nach Lubin et al. 2013)\nklinisch aktuell: Verschwommensehen bds angegeben, Sch...\nOutput:\nschubförmige remittierende Multiple Sklerose\n\nInput:\nMultiple Sklerose mit sekundär progredientem Verlauf seit ca. 2004 (EM 1983, ED 1996)\nklinisch:\naktuell spastische Hemiparese links, stark eingeschränkte Gehstrecke (Rollator, Rollauto für längere Str...\nOutput:\nsekundär progrediente Multiple Sklerose\n\nInput:\nSt. n. Bimaxilläre Retrognathie mit/bei:\nINDENT St. n. Le Fort I Osteotomie und sagittale Spaltung Unterkiefer beidseits am 08.02.2011...\nOutput:\nother\n\n \nPlease provide your answer for the following Input.\nInput:\nSchubförmige Multiple Sklerose (EM, ED 12/2005), EDSS 1.0[/INST] primär progrediente Multiple Sklerose \n"
6,primär progrediente multiple sklerose,primär progrediente Multiple Sklerose,0,True,1,62278320-172E-4D59-AE84-4F28A2ED877D,"Schubförmige Multiple Sklerose (EM und ED 05/2013), EDSS 2.0","[INST]<<SYS>>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.\nYour answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.\nPlease ensure that your responses are socially unbiased and positive in nature.\nIf a question does not makeany sense, or is not factually coherent, explain why instead of answering something not correct. \nIf you don’t know the answer to a question, please don’t share false information.\n<</SYS>>\n\nYour task is to extract the diagnosis corresponding to a type of multiple sclerosis (MS) stated in a German medical report. The input for this task is a German medical report, and the output should be the type of MS.\nThere are three types of multiple sclerosis in German:\n- primär progrediente Multiple Sklerose (PPMS)\n- sekundär progrediente Multiple Sklerose (SPMS)\n- schubförmig remittierende Multiple Sklerose (RRMS)\n\nThe type is provided in the text, and your task is to extract it. If you cannot 

,model_answers,model_answers_mapped,preds,exact_match,labels,rid,text,whole_prompt
19,### output:\nother,other,3,True,1,96BC21AA-235F-4EED-A74F-58EFC11C1176,"Hochgradiger V.a. entzündliche ZNS-Erkrankung ED 03.05.2019, EM 30.04.2019\nINDENT ätiologisch: möglicherweise multiple Sklerose","[INST]<<SYS>>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.\nYour answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.\nPlease ensure that your responses are socially unbiased and positive in nature.\nIf a question does not makeany sense, or is not factually coherent, explain why instead of answering something not correct. \nIf you don’t know the answer to a question, please don’t share false information.\n<</SYS>>\n\n### Instruction:\nYour task is to extract the diagnosis corresponding to a type of multiple sclerosis (MS) stated in a German medical report. The input for this task is a German medical report, and the output should be the type of MS.\nThere are three types of multiple sclerosis in German:\n- primär progrediente Multiple Sklerose (PPMS)\n- sekundär progrediente Multiple Sklerose (SPMS)\n- schubförmig remittierende Multiple Sklerose (RRMS)\n\nThe type is provided in the text, and your task is to extract it. If you cannot match a type exactly, please answer with 'other'.\nYour answer should solely consist of one of the following:\n- primär progrediente Multiple Sklerose\n- sekundär progrediente Multiple Sklerose\n- schubförmige remittierende Multiple Sklerose\n- other\n\nHere are some examples to help you understand the task:\nInput:\nPrimär chronisch progrediente Multiple Sklerose, EM 2008, ED 2012, EDSS 4.0\naktuell: klinisch: stabil, radiologisch: stabil, Progression: ja (nach Lublin et al. 2013)\nINDENT Besserung der Kraft im lin...\nOutput:\nprimär progrediente Multiple Sklerose\n\nInput:\nSchubförmige Multiple Sklerose (EM, ED 12/2005), EDSS 1.0\nklinisch fraglich aktiv, radiologisch aktiv, keine Progression (nach Lubin et al. 2013)\nklinisch aktuell: Verschwommensehen bds angegeben, Sch...\nOutput:\nschubförmige remittierende Multiple Sklerose\n\nInput:\nMultiple Sklerose mit sekundär progredientem Verlauf seit ca. 2004 (EM 1983, ED 1996)\nklinisch:\naktuell spastische Hemiparese links, stark eingeschränkte Gehstrecke (Rollator, Rollauto für längere Str...\nOutput:\nsekundär progrediente Multiple Sklerose\n\nInput:\nSt. n. Bimaxilläre Retrognathie mit/bei:\nINDENT St. n. Le Fort I Osteotomie und sagittale Spaltung Unterkiefer beidseits am 08.02.2011...\nOutput:\nother\n\n\nPlease provide your answer for the following Input.\n\n### Input:\nHochgradiger V.a. entzündliche ZNS-Erkrankung ED 03.05.2019, EM 30.04.2019\nINDENT ätiologisch: möglicherweise multiple Sklerose[/INST]\n\n### Output:\nother \n"
42,### output:\nschubförmige remittierende multiple sklerose,schubförmige remittierende Multiple Sklerose,1,True,2,786DE63D-D0A9-4380-9BD6-3E5EF553376A,"Multiple Sklerose sekundär progredient mit überlagernden Schüben, EM 2005 ED 2006 EDSS 4.0","[INST]<<SYS>>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.\nYour answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.\nPlease ensure that your responses are socially unbiased and positive in nature.\nIf a question does not makeany sense, or is not factually coherent, explain why instead of answering something not correct. \nIf you don’t know the answer to a question, please don’t share false information.\n<</SYS>>\n\n### Instruction:\nYour task is to extract the diagnosis corresponding to a type of multiple sclerosis (MS) stated in a German medical report. The input for this task is a German medical report, and the output should be the type of MS.\nThere are three types of multiple sclerosis in German:\n- primär progrediente Multiple Sklerose (PPMS)\n- sekundär pr

,weighted avg.f1-score,weighted avg.support,strategy,valid_label,accuracy,0.precision,0.recall,0.f1-score,0.support,1.precision,1.recall,1.f1-score,1.support,2.precision,2.recall,2.f1-score,2.support,3.precision,3.recall,3.f1-score,3.support,macro avg.precision,macro avg.recall,macro avg.f1-score,macro avg.support,weighted avg.precision,weighted avg.recall
0,0.922680,58.0,zero_shot_vanilla,0.620690,0.913793,0.500000,1.0,0.666667,4.0,1.000000,0.871795,0.931507,39.0,1.0,1.000000,1.0,3.0,0.923077,1.0,0.960000,12.0,0.855769,0.967949,0.889543,58.0,0.949602,0.913793
0,0.918585,58.0,zero_shot_instruction,0.827586,0.913793,0.571429,1.0,0.727273,4.0,1.000000,0.897436,0.945946,39.0,1.0,0.666667,0.8,3.0,0.857143,1.0,0.923077,12.0,0.857143,0.891026,0.849074,58.0,0.940887,0.913793
0,0.683707,58.0,few_shot_vanilla,0.844828,0.620690,0.160000,1.0,0.275862,4.0,1.000000,0.461538,0.631579,39.0,1.0,0.666667,0.8,3.0,0.923077,1.0,0.960000,12.0,0.770769,0.782051,0.666860,58.0,0.926154,0.620690
0,0.947542,58.0,few_shot_instruction,0.931034,0.948276,1.000000,1.0,1.000000,4.0,0.973684,0.948718,0.961039,39.0,1.0,0.666667,0.8,3.0,0.857143,1.0,0.923077,12.0,0.957707,0.903846,0.921029,58.0,0.952748,0.948276


## Outlines

In [10]:
pd.set_option('display.max_columns', None)
summarize_performance(["ms-diag_outlines_Llama2-MedTuned-13b_4bit_all_test_zero_shot_vanilla.pt",
                       "ms-diag_outlines_Llama2-MedTuned-13b_4bit_all_test_zero_shot_instruction.pt",
                       "ms-diag_outlines_Llama2-MedTuned-13b_4bit_all_test_few_shot_vanilla.pt",
                       "ms-diag_outlines_Llama2-MedTuned-13b_4bit_all_test_few_shot_instruction.pt",
                       ], labels="label_encodings_Llama2-MedTuned-7b.pt", load_line=False, load_outlines=True, verbose = True)

,model_answers,preds,exact_match,labels,rid,text
0,primär progrediente Multiple Sklerose,0,True,1,55E636B5-EA14-497D-832B-A87444F3C096,"Schubförmige Multiple Sklerose (EM, ED 12/2005), EDSS 1.0\nklinisch fraglich aktiv, radiologisch aktiv, keine Progression (nach Lubin et al. 2013)\nklinisch aktuell: Verschwommensehen bds angegeben, Schwankschwindel angegeben, sonst kein fokal-neurologisches Defizit\nVerlauf:\n12/2005 Hemisyndrom links, Dysarthrie, Dysästhesie\n04/2014 Optikusneuritis rechts \n10/2017 sensibles Hemisyndrom links\n02/2020: Zunahme des bekannten Schwankschwindels, zusätzlich mono- und binokuläres Verschwommensehen bds, Visus 0.88 bds, sonst kein fokal-neurologisches Defizit DD leichter Schub im Rahmen der Läsion im Bereich der Radiatio optica DD Uthoff Phänomen i.R. eines viralen Infektes\nDiagnostisch:\n12/2005: Auswärtiges MRI vom 14.12.05: Multiple ZNS Läsionen, eine KM-anreichernd in der Medulla oblongata rechts.\n12/2005 Liquor: Zellzahl und Protein normal, OKB positiv\n12/2005 VEP auswärtig (Dr. Riederer): normal\n05/2014 MRI Hirn: seit 03/2006 zeitliche Dissemination der nicht KM-anreichernden, demyelinisierenden Läsionen periventrikulär, subkortikal und infratentoriell mit/bei Neuritis N. optici rechts.\n22.05.2015 cMRI: stationärer Befund\n03.06.2016 cMRI: Eine neue supratentorielle Läsion in / an der Forceps major links. Zudem eine fraglich neue punktförmige Läsion im cervicalen Myelon rechts.\n03/2017 cMRI: 3 neue Läsionen, keine KM-Aufnahme\n16.10.2017 cMRI: neue Läsionen, keine KM-Aufnahme\n18.04.2018 cMRI: eine neue Gd+ Läsion\n12.02.2020 cMRI: drei neue Gd+ Läsionen, eine neue Gd- Läsion (alle supratentoriell)\nTherapie: \n17.04.-21.04.2014 Methylprednisolon-Stosstherapie \nBislang keine immunmodulatorische Therapie\n15-19.10.2017: Solu-Medrol-Stoss-Therapie 1000mg/d\nTherapie mit Tysabri (bei negativem JCV-Status) oder Ocrevus (bei positivem JCV-Status) geplant"
1,primär progrediente Multiple Sklerose,0,True,1,CE90E38D-BBBF-4F7B-98E0-B87EE0E6DFCE,"Schubförmige Multiple Sklerose, EM 05/2018, ED 07/2018\nINDENT EDSS 0.0\nINDENT Aktuell: klinisch nicht aktiv, radiologisch nicht aktiv, keine Progression (nach Lublin 2013)\nINDENT Verlauf: \nINDENT 03.05.2018: Kribbelparästhesie im linken Fuss dorsalseitig und plantar, sowie intermittierend (< 24h) im linken Unterschenkel und im Verlauf linke palmare Handfläche\nINDENT 29.06.2018: Kribbelparästhesien der rechten Wade, Krafteinschränkung der Beine, Gehstrecke auf 500-700 m eingeschränkt, Müdigkeit, Erschöpfbarkeit\nINDENT 10.08.2018: vor ca. 2 Wochen aufsteigende Progredienz der bekannten Kribbelparästhesien, Hypästhesie bis Höhe des Bauchnabels. Vorstellung in einer Klinik in Belgrad, dort Kortisonstoss, anschliessend Regredienz der Hypästhesien, allerdings neu augetretene Sehstörungen, sowie Verstärkung einer vorbekannten Urge-Inkontinenz.\nINDENT 08/2018: Visus re. 0.9, reduzierte Sensibilität für alle Modalitäten sub TH5 Niveau, proximal betonte Paraparese, Ataxie OEX rechts und UEX rechtsbetont, ataktisches Gangbild\nINDENT Labor: \nINDENT Liquor 06/2018: Zellzahl 21/µl, OKBs im Liquor positiv, im Serum negativ; keine intrathekale IgM-/IgA-Synthese\nINDENT Immunologisches Screening: Antinukleäre AK 1:320 (Anti-Chromatin/Histon/DFS70 negativ), ansonsten unauffällig \nINDENT Anti-NMO und Anti-MOG negativ\nINDENT Infektiologisches Schreening: negativ (HBV, HCV, HIV, Treponema pallidum, Borrelia burgdorferi)\nINDENT Anti-JCV-AK am 27.06.2018: 3,13\nINDENT Bildgebung:\nINDENT cMRI 06/2020: Stationäre intrakranielle Läsionslast, keine neue oder KM-affine Läsion. \nINDENT Therapie: \nINDENT 07/2018: orale Cortisonstosstherapie mit Medrol 1000 mg/d \nINDENT 07/2018: i. v. Cortisonstosstherapie 500mg/d (Klinik Belgrad, Wirkstoff unbekannt), 1000mg/d \nINDENT 08/2018: i. v. Cortisonstosstherapie 2000mg/d \nINDENT Seit 08/2018: Ocrelizumab"
3,primär progrediente Multiple Sklerose,0,True,1,69273DEA-ACAC-40CD-8F02-840EB3CC2027,"Schubförmig-remittierende Multip

model_answers  preds  exact_match  labels  \
25                                  other      3         True       1   
26  primär progrediente Multiple Sklerose      0         True       3   
30                                  other      3         True       0   
44  primär progrediente Multiple Sklerose      0         True       1   
49  primär progrediente Multiple Sklerose      0         True       1   
56  primär progrediente Multiple Sklerose      0         True       1   

                                     rid  \
25  96BC21AA-235F-4EED-A74F-58EFC11C1176   
26  1ED19CB8-D11F-483B-ACC5-932BED8C5430   
30  11A4453B-76B6-4558-8126-EEFEA6935C91   
44  54FBE2C1-4F4A-4E43-AD9B-CF5774675F73   
49  BE04513F-6108-4398-926A-724146897A64   
56  B5B6D014-7E02-44E5-8390-F571F7C1D4E5   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

,model_answers,preds,exact_match,labels,rid,text
0,primär progrediente Multiple Sklerose,0,True,1,55E636B5-EA14-497D-832B-A87444F3C096,"Schubförmige Multiple Sklerose (EM, ED 12/2005), EDSS 1.0\nklinisch fraglich aktiv, radiologisch aktiv, keine Progression (nach Lubin et al. 2013)\nklinisch aktuell: Verschwommensehen bds angegeben, Schwankschwindel angegeben, sonst kein fokal-neurologisches Defizit\nVerlauf:\n12/2005 Hemisyndrom links, Dysarthrie, Dysästhesie\n04/2014 Optikusneuritis rechts \n10/2017 sensibles Hemisyndrom links\n02/2020: Zunahme des bekannten Schwankschwindels, zusätzlich mono- und binokuläres Verschwommensehen bds, Visus 0.88 bds, sonst kein fokal-neurologisches Defizit DD leichter Schub im Rahmen der Läsion im Bereich der Radiatio optica DD Uthoff Phänomen i.R. eines viralen Infektes\nDiagnostisch:\n12/2005: Auswärtiges MRI vom 14.12.05: Multiple ZNS Läsionen, eine KM-anreichernd in der Medulla oblongata rechts.\n12/2005 Liquor: Zellzahl und Protein normal, OKB positiv\n12/2005 VEP auswärtig (Dr. Riederer): normal\n05/2014 MRI Hirn: seit 03/2006 zeitliche Dissemination der nicht KM-anreichernden, demyelinisierenden Läsionen periventrikulär, subkortikal und infratentoriell mit/bei Neuritis N. optici rechts.\n22.05.2015 cMRI: stationärer Befund\n03.06.2016 cMRI: Eine neue supratentorielle Läsion in / an der Forceps major links. Zudem eine fraglich neue punktförmige Läsion im cervicalen Myelon rechts.\n03/2017 cMRI: 3 neue Läsionen, keine KM-Aufnahme\n16.10.2017 cMRI: neue Läsionen, keine KM-Aufnahme\n18.04.2018 cMRI: eine neue Gd+ Läsion\n12.02.2020 cMRI: drei neue Gd+ Läsionen, eine neue Gd- Läsion (alle supratentoriell)\nTherapie: \n17.04.-21.04.2014 Methylprednisolon-Stosstherapie \nBislang keine immunmodulatorische Therapie\n15-19.10.2017: Solu-Medrol-Stoss-Therapie 1000mg/d\nTherapie mit Tysabri (bei negativem JCV-Status) oder Ocrevus (bei positivem JCV-Status) geplant"
3,primär progrediente Multiple Sklerose,0,True,1,69273DEA-ACAC-40CD-8F02-840EB3CC2027,"Schubförmig-remittierende Multiple Sklerose, EM und ED 04/2007\nINDENT Diagnostik\nINDENT EDSS 05/2019: 2.0\nINDENT MRI-Neuroachse 08/2018: Intrakraniell sowie spinal kein Nachweis von neuen oder KM-aufnehmenden Läsionen\nINDENT MRI Schädel vom 08/2013: 1 neue gliotische Läsion im Übergang Mittelhirn zu oberem Kleinhirnschenkel der linken Seite\nINDENT MRI Neuroachse vom 09/2012: Fraglich neue Läsion im Centrum semiovale post rechts, ansonsten stabiler Befund zerebral. Spinal fragliche Progredienz der Läsionen des thorakalen Myelons\nINDENT Sensomotorisches Querschnittsyndrom, sensibel: sub-Th1, rechts-betont sowie distal-betont an allen Extremitäten, motorische Parese Kleinfinger rechts 04/2007\nINDENT MRI Schädel vom 04/2007: ausgedehnt KM-aufnehmende Läsion C1-3, 2 kleine Läsionen obere u. untere BWS; multiple (> 20), nicht KM-aufnehmende periventrikuläre T2-Läsionen \nINDENT Liquor 04/2007: Zellzahl 8/µlL Protein 0,48 g/L, IgG-Index 1.44, OKB positiv\nINDENT Therapie:\nINDENT aktuell: Teriflunomid (Aubagio) seit 05/2018\nINDENT St.n. Plegridy 09/2015 - 05/2018\nINDENT St.n. Tecfidera Therapie 05. - 06/2016\nINDENT St.n. immunmodulatorische Therapie mit Betaferon 09/2007 - 03/2015\nINDENT St.n. Solu-Medrol 3x 1 g 04/2007"
4,primär progrediente Multiple Sklerose,0,True,1,83D5B33F-2967-40C9-BA12-7173CF6C1BE0,"Multiple Sklerose, schubförmiger Verlauf (EM 2006, ED 06.08.2014)\nINDENT Verlauf:\nINDENT 2006: Koordinationsstörung der rechten Körperhälfte\nINDENT 05/2014 Parästhesien der Extemitäten li>re und Gesicht seit 05/2014\nINDENT Diagnostik\nINDENT laborchemisch 10/06: TSH, MAJ, TAK, Eiweisselektrophorese, C3- C4 Komplement, ACE normal\nINDENT Serologie 10/06: Coxsackie-Virus- AL der Serotypen 1, 2, 6, Enteroviren AK; ANA : erhöht, HAV, HBV,HCV, CMV,HSV, und Luesserologie negativ, Vaskulitsserologie: unauffällig\nINDENT Liquor vom 10/06: 6 mononukleäre Zellen/µl, Protein193 mg/l, OKB positiv\nINDENT EEG vom 10/06: unauffällig\nINDENT Röntgen Thorax 1

,model_answers,preds,exact_match,labels,rid,text
25,other,3,True,1,96BC21AA-235F-4EED-A74F-58EFC11C1176,"Hochgradiger V.a. entzündliche ZNS-Erkrankung ED 03.05.2019, EM 30.04.2019\nINDENT ätiologisch: möglicherweise multiple Sklerose\nINDENT anamnestisch: seit 4 Tagen aufsteigende Dysästhesie von den Füssen bis etwa T12, distal betont; St. n. Atemwegsinfektion vor einigen Tagen\nINDENT klinisch: Dysästhesie sub Th12, Hypopallästhesie malleolar bds 7/8, Lagesinn und Temperatursinn erhalten, keine Visusstörungen, leicht gesteigerte Reflexe (BSR und PSR bds) \nINDENT diagnostisch\nINDENT MRI BWS/HWS 03.05.2019: Intraspinale intramedulläre flau T2-hyperintense, kontrastmittelaufnehmende Läsion auf Höhe HWK1/HWK2 mit hochgradiger Verdacht auf Myelitis. Weitere T2-hyperintense intramedulläre Signalanhebungen auf Höhe BWK5 - BWK8 ohne Kontrastmittelaufnahme.\nINDENT LP 03.05.2019: Zellzahl 11, Protein 0,565 g/l, OKB positiv (5 Banden)\nINDENT Labor: AQP-4-Ak und Vaskulitis-Screening im Serum negativ\nINDENT Urin-Status 03.05.2019: blande\nINDENT cMRI vom 06.05.2019: Gliotischer Defekt des Crus posterior der Capsula interna links\nINDENT Elektrophysiologie vom 07.05.2019: normale Tibialis-SEPs.\nINDENT therapeutisch\nINDENT Kortisonstoss mit Solu-Medrol 1g/d i.v. vom 04.05-08.05.2019"
37,other,3,True,1,ABF47F0B-D5E3-46A7-B506-1B961357E158,"Chronisch-demyelinisierende ZNS-Erkrankung, EM 01/2016, ED 02/2018\nINDENT ätiologisch: a.e. schubförmige Multiple Sklerose (McDonald Kriterien 2010 erfüllt, Ausschlussdiagnostik noch nicht komplettiert)\nINDENT anamestisch: \nINDENT aktuell: ungerichteter Schwindel, Doppelbilder bei Blick nach rechts, Nausea am 01.03.2018, Fazialisparese links seit 03.03.2018\nINDENT Sommer 2017: pins-and-needles auf halber Körperseite für ca. 1-2 Tage\nINDENT Januar 2016: Dysarthrie, Koordinationsstörungen, Fatigue für 2 Tage\nINDENT klinisch: Komplexe Okulomotorikstörung mit INO links, Trochlearisparese links, Blickrichtungsnystagmus horizontal und vertikal, Fazialisparese links (a.e. nukleär)Reflexbetonung rechts.\nINDENT diagnostisch:\nINDENT cCT-Angio 02.03.2018: keine Blutung keine Gefässstenose\nINDENT cMRI vom 02.03.2018: Multiple FLAIR Hyperintensitäten supratentoriell (juxtakortikal, periventrikulär, Marklager), am linken Hinterhorn (12mm) mit KM Aufnahme, Gyrus frontalis sup. und medius rechts 9mm mit KM Aufnahme, Capsula interna links (7mm), Pons links (7mm), weitere Läsionen frontal, temporal, parietal links, okziptal beidseits. Nervi optici regelrecht.\nINDENT Liquor vom 02.03.2018: klar, Protein 0,28 g/l und Zellzahl 2/µl regelrecht, OKB ausstehend\nINDENT Labor extern Stadt Spital Waid: Serologie Aquaporin 4 (USZ) CMV (USZ) HSV Typ 1 und 2 (USZ). HZV (USZ). VZV (USZ), Borellien und Lues im Liquor (UZH)\nINDENT Vaskulitisscreening, Vitamin D und B12 ausstehend \nINDENT therapeutisch: \nINDENT Solumedrol 1g i.v 02-06.03.2018 (Stadtspital Waid)"
44,primär progrediente Multiple Sklerose,0,True,1,54FBE2C1-4F4A-4E43-AD9B-CF5774675F73,"INDENT 39-jährige Patientin mit primärer Sterilität seit 15 Jahren (1998)\nINDENT Multiple Sklerose, schubförmiger Verlauf mit Residuen (ED: 9/2004)\nINDENT Beim Partner Verdacht auf Subfertilität bei hypo- bis normogonadotropem Hypogonadismus, Status nach off label use Clomifen 50 mg jeden 2. Tag und anschliessender probatorischer Spermienaufbereitung"
56,primär progrediente Multiple Sklerose,0,True,1,B5B6D014-7E02-44E5-8390-F571F7C1D4E5,"Multiple Sklerose (EM 05/2011, ED 09/2011)\nINDENT aktuell: EDSS 2.5 residuell gelegentlicher Schwankschwindel bei schnellen Bewegungen und Lagewechseln.\nINDENT Verlauf:\nINDENT 05. 2011: Schub mit schleichendem Beginn im Mai 2011 mit Doppelbildern und Schwankschwindel, leichter Gangataxie, Kribbelparästhesien Dig V links. Deutliche Regredienz der Symptome auf Solumedrol (08/2011)\nINDENT diagnostisch:\nINDENT 10.8.11: VEP pds opB\nINDENT 10.8.11 MRI Schädel: multiple entzündliche Läsionen juxtacortical, periventrikulär, infratentoriell, 

,weighted avg.f1-score,weighted avg.support,strategy,valid_label,accuracy,0.precision,0.recall,0.f1-score,0.support,1.precision,1.recall,1.f1-score,1.support,2.precision,2.recall,2.f1-score,2.support,3.precision,3.recall,3.f1-score,3.support,macro avg.precision,macro avg.recall,macro avg.f1-score,macro avg.support,weighted avg.precision,weighted avg.recall
0,0.101649,58.0,zero_shot_vanilla,1.0,0.120690,0.095238,1.00,0.173913,4.0,0.500000,0.076923,0.133333,39.0,0.0,0.0,0.0,3.0,0.000000,0.000000,0.000000,12.0,0.148810,0.269231,0.076812,58.0,0.342775,0.120690
0,0.907478,58.0,zero_shot_instruction,1.0,0.896552,0.428571,0.75,0.545455,4.0,1.000000,0.897436,0.945946,39.0,1.0,1.0,1.0,3.0,0.846154,0.916667,0.880000,12.0,0.818681,0.891026,0.842850,58.0,0.928761,0.896552
0,0.270360,58.0,few_shot_vanilla,1.0,0.241379,0.093023,1.00,0.170213,4.0,0.769231,0.256410,0.384615,39.0,0.0,0.0,0.0,3.0,0.000000,0.000000,0.000000,12.0,0.215564,0.314103,0.138707,58.0,0.523657,0.241379
0,0.933945,58.0,few_shot_instruction,1.0,0.931034,0.666667,1.00,0.800000,4.0,1.000000,0.897436,0.945946,39.0,1.0,1.0,1.0,3.0,0.857143,1.000000,0.923077,12.0,0.880952,0.974359,0.917256,58.0,0.947455,0.931034


## RAG + Outlines

In [11]:
pd.set_option('display.max_columns', None)
summarize_performance(["ms-diag_outlines_Llama2-MedTuned-13b_4bit_all_test_zero_shot_vanilla_rag.pt",
                       "ms-diag_outlines_Llama2-MedTuned-13b_4bit_all_test_zero_shot_instruction_rag.pt",
                       "ms-diag_outlines_Llama2-MedTuned-13b_4bit_all_test_few_shot_vanilla_rag.pt",
                       "ms-diag_outlines_Llama2-MedTuned-13b_4bit_all_test_few_shot_instruction_rag.pt",
                       ], labels="label_encodings_Llama2-MedTuned-13b.pt", load_line=False, load_outlines=True, verbose = True)

,model_answers,preds,exact_match,labels,rid,text
0,sekundär progrediente Multiple Sklerose,2,True,1,55E636B5-EA14-497D-832B-A87444F3C096,"Schubförmige Multiple Sklerose (EM, ED 12/2005), EDSS 1.0"
1,sekundär progrediente Multiple Sklerose,2,True,1,CE90E38D-BBBF-4F7B-98E0-B87EE0E6DFCE,"Schubförmige Multiple Sklerose, EM 05/2018, ED 07/2018"
2,sekundär progrediente Multiple Sklerose,2,True,0,B0B07B18-0211-4467-8489-581AC761D297,"Primär chronisch progrediente Multiple Sklerose, EM 2008, ED 2012, EDSS 4.0"
3,sekundär progrediente Multiple Sklerose,2,True,1,69273DEA-ACAC-40CD-8F02-840EB3CC2027,"Schubförmig-remittierende Multiple Sklerose, EM und ED 04/2007\nINDENT EDSS 05/2019: 2.0"
4,sekundär progrediente Multiple Sklerose,2,True,1,83D5B33F-2967-40C9-BA12-7173CF6C1BE0,"Multiple Sklerose, schubförmiger Verlauf (EM 2006, ED 06.08.2014)"
6,sekundär progrediente Multiple Sklerose,2,True,1,62278320-172E-4D59-AE84-4F28A2ED877D,"Schubförmige Multiple Sklerose (EM und ED 05/2013), EDSS 2.0"
7,sekundär progrediente Multiple Sklerose,2,True,1,82C11DBE-09F4-4C2C-8F6F-B1D75BF9E790,Schubförmige multiple Sklerose (ED Januar 2013)
9,sekundär progrediente Multiple Sklerose,2,True,1,08F1FA27-6213-47AF-B967-DC44CCCE9613,"Schubförmig remittierende Multiple Sklerose (EM 09/14), EDSS 0.0"
10,primär progrediente Multiple Sklerose,0,True,1,C545D8B8-9F4A-4628-B1DA-5F2FD64F1008,"Schubförmige Multiple Sklerose (EM 05/2015, ED 07/2015)\nEDSS: 2.0 (Pyz: 1.0; Sens: 2.0; Cerebral/Fatigue: 2.0)"
11,primär progrediente Multiple Sklerose,0,True,1,22719E9F-BE4F-4C86-A728-EC6B1AF64B8F,"Schubförmige Multiple Sklerose (EM 04/2010, ED 06/2010), EDSS: 2.0"


,model_answers,preds,exact_match,labels,rid,text
19,other,3,True,1,96BC21AA-235F-4EED-A74F-58EFC11C1176,"Hochgradiger V.a. entzündliche ZNS-Erkrankung ED 03.05.2019, EM 30.04.2019\nINDENT ätiologisch: möglicherweise multiple Sklerose"
44,other,3,True,1,B5B6D014-7E02-44E5-8390-F571F7C1D4E5,"Multiple Sklerose (EM 05/2011, ED 09/2011)"


,model_answers,preds,exact_match,labels,rid,text
1,primär progrediente Multiple Sklerose,0,True,1,CE90E38D-BBBF-4F7B-98E0-B87EE0E6DFCE,"Schubförmige Multiple Sklerose, EM 05/2018, ED 07/2018"
21,primär progrediente Multiple Sklerose,0,True,1,107AB593-34BD-4329-8A9A-D3584819FE50,Multiple Sklerose mit vorherrschend schubförmigem Verlauf: Ohne Angabe einer akuten Exazerbation oder Progression
23,primär progrediente Multiple Sklerose,0,True,1,086D33C8-DDD3-4DE2-BFE0-A945076A2335,"Schubförmige Multiple Sklerose (EM 12/2009, ED 09/2010), aktueller EDSS 0"
25,primär progrediente Multiple Sklerose,0,True,1,1914ADB8-2E2D-4557-930E-4AB8422CBD5F,"Vd.a. autoimmun-entzündliche ZNS-Ekrankung, am ehesten einer schubförmig-remittierenden multiplen Sklerose entsprechend (EM 1997):"
27,primär progrediente Multiple Sklerose,0,True,1,659D3D0B-AD43-43D4-A99A-34891BA3B676,"Schubförmige Multiple Sklerose, ED 07/2016"
28,sekundär progrediente Multiple Sklerose,2,True,1,ABF47F0B-D5E3-46A7-B506-1B961357E158,"Chronisch-demyelinisierende ZNS-Erkrankung, EM 01/2016, ED 02/2018\nINDENT ätiologisch: a.e. schubförmige Multiple Sklerose (McDonald Kriterien 2010 erfüllt, Ausschlussdiagnostik noch nicht komplettiert)"
30,primär progrediente Multiple Sklerose,0,True,1,E71ECC99-2B1A-4061-BDC3-F5FBC039DBC9,"Schubförmige Multiple Sklerose mit Residuen, EM 2000, ED 2006, EDSS 6.5"
32,primär progrediente Multiple Sklerose,0,True,2,E81AD81E-7504-4FFE-AA11-77E288E77CEC,"Multiple Sklerose mit sekundär progredientem Verlauf seit ca. 2004 (EM 1983, ED 1996)"
35,schubförmige remittierende Multiple Sklerose,1,True,2,763EC115-4945-4536-A788-A1CC7B8A6A79,"Sekundär progrediente Multiple Sklerose (EM 1998, ED 1999), EDSS 7.0"
36,primär progrediente Multiple Sklerose,0,True,1,1B0FEA22-621B-4D5D-A42B-D0E53FD1AECD,"Schubförmige Multiple Sklerose (ED 01/2013), EDSS 0"


,model_answers,preds,exact_match,labels,rid,text
19,other,3,True,1,96BC21AA-235F-4EED-A74F-58EFC11C1176,"Hochgradiger V.a. entzündliche ZNS-Erkrankung ED 03.05.2019, EM 30.04.2019\nINDENT ätiologisch: möglicherweise multiple Sklerose"
42,schubförmige remittierende Multiple Sklerose,1,True,2,786DE63D-D0A9-4380-9BD6-3E5EF553376A,"Multiple Sklerose sekundär progredient mit überlagernden Schüben, EM 2005 ED 2006 EDSS 4.0"
44,other,3,True,1,B5B6D014-7E02-44E5-8390-F571F7C1D4E5,"Multiple Sklerose (EM 05/2011, ED 09/2011)"


,weighted avg.f1-score,weighted avg.support,strategy,valid_label,accuracy,0.precision,0.recall,0.f1-score,0.support,1.precision,1.recall,1.f1-score,1.support,2.precision,2.recall,2.f1-score,2.support,3.precision,3.recall,3.f1-score,3.support,macro avg.precision,macro avg.recall,macro avg.f1-score,macro avg.support,weighted avg.precision,weighted avg.recall
0,0.263187,58.0,zero_shot_vanilla,1.0,0.310345,0.250000,0.5,0.333333,4.0,1.000000,0.025641,0.050000,39.0,0.083333,1.000000,0.153846,3.0,0.923077,1.0,0.960000,12.0,0.564103,0.631410,0.374295,58.0,0.884947,0.310345
0,0.966390,58.0,zero_shot_instruction,1.0,0.965517,1.000000,1.0,1.000000,4.0,1.000000,0.948718,0.973684,39.0,1.000000,1.000000,1.000000,3.0,0.857143,1.0,0.923077,12.0,0.964286,0.987179,0.974190,58.0,0.970443,0.965517
0,0.774999,58.0,few_shot_vanilla,1.0,0.758621,0.285714,1.0,0.444444,4.0,0.933333,0.717949,0.811594,39.0,0.000000,0.000000,0.000000,3.0,0.923077,1.0,0.960000,12.0,0.535531,0.679487,0.554010,58.0,0.838272,0.758621
0,0.947542,58.0,few_shot_instruction,1.0,0.948276,1.000000,1.0,1.000000,4.0,0.973684,0.948718,0.961039,39.0,1.000000,0.666667,0.800000,3.0,0.857143,1.0,0.923077,12.0,0.957707,0.903846,0.921029,58.0,0.952748,0.948276


### Label Corrections
- rid: 96BC21AA-235F-4EED-A74F-58EFC11C1176, 
    - text: Hochgradiger V.a. entzündliche ZNS-Erkrankung ED 03.05.2019, EM 30.04.2019\nINDENT ätiologisch: möglicherweise multiple Sklerose
    - label: 1
    - corrected label: 3
- rid: B5B6D014-7E02-44E5-8390-F571F7C1D4E5
    - text: Multiple Sklerose (EM 05/2011, ED 09/2011)
    - label: 1
    - corrected label: 3

In [6]:
label_correction = {"96BC21AA-235F-4EED-A74F-58EFC11C1176" : 3, "B5B6D014-7E02-44E5-8390-F571F7C1D4E5": 3}

In [21]:
overall_results = pd.concat([summarize_performance(["ms-diag_Llama2-MedTuned-13b_4bit_all_test_zero_shot_vanilla.pt",
                                                       "ms-diag_Llama2-MedTuned-13b_4bit_all_test_zero_shot_instruction.pt",
                                                       "ms-diag_Llama2-MedTuned-13b_4bit_all_test_few_shot_vanilla.pt",
                                                       "ms-diag_Llama2-MedTuned-13b_4bit_all_test_few_shot_instruction.pt",
                                                       ], labels="label_encodings_Llama2-MedTuned-13b.pt", load_line=False, load_outlines=False, label_correction=label_correction),
                              summarize_performance(["ms-diag_Llama2-MedTuned-13b_4bit_all_test_zero_shot_vanilla_rag.pt",
                                                      "ms-diag_Llama2-MedTuned-13b_4bit_all_test_zero_shot_instruction_rag.pt",
                                                         "ms-diag_Llama2-MedTuned-13b_4bit_all_test_few_shot_vanilla_rag.pt",
                                                         "ms-diag_Llama2-MedTuned-13b_4bit_all_test_few_shot_instruction_rag.pt",
                                                         ], labels="label_encodings_Llama2-MedTuned-13b.pt", load_line=False, load_outlines=False, label_correction=label_correction),
                            summarize_performance(["ms-diag_outlines_Llama2-MedTuned-13b_4bit_all_test_zero_shot_vanilla.pt",
                                                       "ms-diag_outlines_Llama2-MedTuned-13b_4bit_all_test_zero_shot_instruction.pt",
                                                       "ms-diag_outlines_Llama2-MedTuned-13b_4bit_all_test_few_shot_vanilla.pt",
                                                       "ms-diag_outlines_Llama2-MedTuned-13b_4bit_all_test_few_shot_instruction.pt",
                                                       ], labels="label_encodings_Llama2-MedTuned-7b.pt", load_line=False, load_outlines=True, label_correction=label_correction),
                            summarize_performance(["ms-diag_outlines_Llama2-MedTuned-13b_4bit_all_test_zero_shot_vanilla_rag.pt",
                                                       "ms-diag_outlines_Llama2-MedTuned-13b_4bit_all_test_zero_shot_instruction_rag.pt",
                                                       "ms-diag_outlines_Llama2-MedTuned-13b_4bit_all_test_few_shot_vanilla_rag.pt",
                                                       "ms-diag_outlines_Llama2-MedTuned-13b_4bit_all_test_few_shot_instruction_rag.pt",
                                                       ], labels="label_encodings_Llama2-MedTuned-13b.pt", load_line=False, load_outlines=True, label_correction=label_correction)], ignore_index=True)
overall_results["approach"] = ["base"] * 4 + ["rag"] * 4 + ["outlines"] * 4 + ["outlines_rag"] * 4
overall_results.rename(columns={"macro avg.f1-score": "Macro F1-Score",
                                "macro avg.precision": "Macro Precision",
                                "macro avg.recall": "Macro Recall"}, inplace=True)

In [22]:
print(generate_latex_table(overall_results, 
                           caption="Macro Precision, Recall and F1-Score for different prompting strategies using Llama2-MedTuned-13B \cite{rohanian2023exploring}",
                           label="tab:ms-diag-13B",
                           metrics=["Macro Precision", "Macro Recall", "Macro F1-Score"]))

\begin{table}[h]
    \centering
    \begin{tabular}{lcccc}
        \toprule
        & \multicolumn{4}{c}{Macro Precision}\\
        \cmidrule(lr){2-5}
        & Base & +S2A & +Outlines & +S2A \& Outlines \\
        \midrule
        zero-shot-vanilla & 0.56 & 0.88 & 0.15 & 0.56 \\
        zero-shot-instruction & 0.79 & 0.89 & 0.84 & 1.00 \\
        few-shot-vanilla & 0.44 & 0.79 & 0.20 & 0.52 \\
        few-shot-instruction & 0.88 & 0.99 & 0.90 & 0.99 \\
        \bottomrule
    \end{tabular}

    \begin{tabular}{lcccc}
        \toprule
        & \multicolumn{4}{c}{Macro Recall}\\
        \cmidrule(lr){2-5}
        & Base & +S2A & +Outlines & +S2A \& Outlines \\
        \midrule
        zero-shot-vanilla & 0.42 & 0.96 & 0.27 & 0.60 \\
        zero-shot-instruction & 0.77 & 0.90 & 0.89 & 1.00 \\
        few-shot-vanilla & 0.47 & 0.77 & 0.31 & 0.64 \\
        few-shot-instruction & 0.91 & 0.92 & 0.97 & 0.92 \\
        \bottomrule
    \end{tabular}

    \begin{tabular}{lcccc}
        \topru

In [23]:
overall_results.to_csv(paths.THESIS_PATH/"ms_pred_results_prompt13b.csv")

## Valid Label

In [9]:
overall_results[['approach', 'strategy', 'valid_label']]

,approach,strategy,valid_label
0,base,zero_shot_vanilla,0.310345
1,base,zero_shot_instruction,0.655172
2,base,few_shot_vanilla,0.689655
3,base,few_shot_instruction,0.965517
4,rag,zero_shot_vanilla,0.620690
5,rag,zero_shot_instruction,0.827586
6,rag,few_shot_vanilla,0.844828
7,rag,few_shot_instruction,0.931034
8,outlines,zero_shot_vanilla,1.000000
9,outlines,zero_shot_instruction,1.000000


# Llama 7B

## Base

In [10]:
pd.set_option('display.max_columns', None)
summarize_performance(["ms-diag_Llama2-MedTuned-7b_4bit_all_test_zero_shot_vanilla.pt",
                       "ms-diag_Llama2-MedTuned-7b_4bit_all_test_zero_shot_instruction.pt",
                       "ms-diag_Llama2-MedTuned-7b_4bit_all_test_few_shot_vanilla.pt",
                       "ms-diag_Llama2-MedTuned-7b_4bit_all_test_few_shot_instruction.pt",
                       ], labels="label_encodings_Llama2-MedTuned-7b.pt", load_line=False, load_outlines=False)

,weighted avg.f1-score,weighted avg.support,strategy,valid_label,accuracy,0.precision,0.recall,0.f1-score,0.support,1.precision,1.recall,1.f1-score,1.support,2.precision,2.recall,2.f1-score,2.support,3.precision,3.recall,3.f1-score,3.support,macro avg.precision,macro avg.recall,macro avg.f1-score,macro avg.support,weighted avg.precision,weighted avg.recall
0,0.358741,58.0,zero_shot_vanilla,0.258621,0.310345,0.125000,0.50,0.200000,4.0,0.888889,0.205128,0.333333,39.0,0.086957,0.666667,0.153846,3.0,0.6,0.500000,0.545455,12.0,0.425211,0.467949,0.308159,58.0,0.734958,0.310345
0,0.096026,58.0,zero_shot_instruction,0.172414,0.068966,0.000000,0.00,0.000000,4.0,0.750000,0.076923,0.139535,39.0,0.022727,0.333333,0.042553,3.0,0.0,0.000000,0.000000,12.0,0.193182,0.102564,0.045522,58.0,0.505486,0.068966
0,0.241462,58.0,few_shot_vanilla,0.137931,0.189655,0.111111,0.25,0.153846,4.0,1.000000,0.128205,0.227273,39.0,0.029412,0.333333,0.054054,3.0,0.4,0.333333,0.363636,12.0,0.385131,0.261218,0.199702,58.0,0.764357,0.189655
0,0.036495,58.0,few_shot_instruction,0.551724,0.051724,0.000000,0.00,0.000000,4.0,0.500000,0.025641,0.048780,39.0,0.037736,0.666667,0.071429,3.0,0.0,0.000000,0.000000,12.0,0.134434,0.173077,0.030052,58.0,0.338159,0.051724


## Outlines

In [11]:
pd.set_option('display.max_columns', None)
summarize_performance(["ms-diag_outlines_Llama2-MedTuned-7b_4bit_all_test_zero_shot_vanilla.pt",
                       "ms-diag_outlines_Llama2-MedTuned-7b_4bit_all_test_zero_shot_instruction.pt",
                       "ms-diag_outlines_Llama2-MedTuned-7b_4bit_all_test_few_shot_vanilla.pt",
                       "ms-diag_outlines_Llama2-MedTuned-7b_4bit_all_test_few_shot_instruction.pt",
                       ], labels="label_encodings_Llama2-MedTuned-7b.pt", load_line=False, load_outlines=True)

,weighted avg.f1-score,weighted avg.support,strategy,valid_label,accuracy,0.precision,0.recall,0.f1-score,0.support,1.precision,1.recall,1.f1-score,1.support,2.precision,2.recall,2.f1-score,2.support,3.precision,3.recall,3.f1-score,3.support,macro avg.precision,macro avg.recall,macro avg.f1-score,macro avg.support,weighted avg.precision,weighted avg.recall
0,0.169644,58.0,zero_shot_vanilla,1.0,0.189655,0.088235,0.75,0.157895,4.0,1.0,0.076923,0.142857,39.0,0.0,0.000000,0.000000,3.0,0.238095,0.416667,0.303030,12.0,0.331583,0.310897,0.150946,58.0,0.727760,0.189655
0,0.730447,58.0,zero_shot_instruction,1.0,0.689655,0.230769,0.75,0.352941,4.0,1.0,0.717949,0.835821,39.0,1.0,0.333333,0.500000,3.0,0.500000,0.666667,0.571429,12.0,0.682692,0.616987,0.565048,58.0,0.843501,0.689655
0,0.248468,58.0,few_shot_vanilla,1.0,0.293103,0.076923,0.25,0.117647,4.0,1.0,0.128205,0.227273,39.0,0.0,0.000000,0.000000,3.0,0.275000,0.916667,0.423077,12.0,0.337981,0.323718,0.191999,58.0,0.734615,0.293103
0,0.770662,58.0,few_shot_instruction,1.0,0.758621,0.800000,1.00,0.888889,4.0,1.0,0.666667,0.800000,39.0,0.5,0.666667,0.571429,3.0,0.521739,1.000000,0.685714,12.0,0.705435,0.833333,0.736508,58.0,0.861394,0.758621


## RAG + Outlines

In [12]:
summarize_performance(["ms-diag_outlines_Llama2-MedTuned-7b_4bit_all_test_zero_shot_vanilla_rag.pt",
                          "ms-diag_outlines_Llama2-MedTuned-7b_4bit_all_test_zero_shot_instruction_rag.pt",
                          "ms-diag_outlines_Llama2-MedTuned-7b_4bit_all_test_few_shot_vanilla_rag.pt",
                          "ms-diag_outlines_Llama2-MedTuned-7b_4bit_all_test_few_shot_instruction_rag.pt",
                          ], labels="label_encodings_Llama2-MedTuned-7b.pt", load_line=False, load_outlines=True)

,weighted avg.f1-score,weighted avg.support,strategy,valid_label,accuracy,0.precision,0.recall,0.f1-score,0.support,1.precision,1.recall,1.f1-score,1.support,2.precision,2.recall,2.f1-score,2.support,3.precision,3.recall,3.f1-score,3.support,macro avg.precision,macro avg.recall,macro avg.f1-score,macro avg.support,weighted avg.precision,weighted avg.recall
0,0.852706,58.0,zero_shot_vanilla,1.0,0.862069,0.444444,1.0,0.615385,4.0,1.0,0.871795,0.931507,39.0,0.0,0.0,0.0,3.0,0.800000,1.0,0.888889,12.0,0.561111,0.717949,0.608945,58.0,0.868582,0.862069
0,0.966390,58.0,zero_shot_instruction,1.0,0.965517,1.000000,1.0,1.000000,4.0,1.0,0.948718,0.973684,39.0,1.0,1.0,1.0,3.0,0.857143,1.0,0.923077,12.0,0.964286,0.987179,0.974190,58.0,0.970443,0.965517
0,0.189990,58.0,few_shot_vanilla,1.0,0.310345,0.222222,1.0,0.363636,4.0,1.0,0.051282,0.097561,39.0,0.0,0.0,0.0,3.0,0.315789,1.0,0.480000,12.0,0.384503,0.512821,0.235299,58.0,0.753075,0.310345
0,0.966390,58.0,few_shot_instruction,1.0,0.965517,1.000000,1.0,1.000000,4.0,1.0,0.948718,0.973684,39.0,1.0,1.0,1.0,3.0,0.857143,1.0,0.923077,12.0,0.964286,0.987179,0.974190,58.0,0.970443,0.965517


In [26]:
overall_results = pd.concat([summarize_performance(["ms-diag_Llama2-MedTuned-7b_4bit_all_test_zero_shot_vanilla.pt",
                            "ms-diag_Llama2-MedTuned-7b_4bit_all_test_zero_shot_instruction.pt",
                            "ms-diag_Llama2-MedTuned-7b_4bit_all_test_few_shot_vanilla.pt",
                            "ms-diag_Llama2-MedTuned-7b_4bit_all_test_few_shot_instruction.pt",
                            ], labels="label_encodings_Llama2-MedTuned-7b.pt", load_line=False, load_outlines=False, label_correction=label_correction),
                                 summarize_performance(["ms-diag_Llama2-MedTuned-7b_4bit_all_test_zero_shot_vanilla_rag.pt",
                            "ms-diag_Llama2-MedTuned-7b_4bit_all_test_zero_shot_instruction_rag.pt",
                            "ms-diag_Llama2-MedTuned-7b_4bit_all_test_few_shot_vanilla_rag.pt",
                            "ms-diag_Llama2-MedTuned-7b_4bit_all_test_few_shot_instruction_rag.pt",
                            ], labels="label_encodings_Llama2-MedTuned-7b.pt", load_line=False, load_outlines=False, label_correction=label_correction),
                                 summarize_performance(["ms-diag_outlines_Llama2-MedTuned-7b_4bit_all_test_zero_shot_vanilla.pt",
                            "ms-diag_outlines_Llama2-MedTuned-7b_4bit_all_test_zero_shot_instruction.pt",
                            "ms-diag_outlines_Llama2-MedTuned-7b_4bit_all_test_few_shot_vanilla.pt",
                            "ms-diag_outlines_Llama2-MedTuned-7b_4bit_all_test_few_shot_instruction.pt",
                            ], labels="label_encodings_Llama2-MedTuned-7b.pt", load_line=False, load_outlines=True, label_correction=label_correction),
                                 summarize_performance(["ms-diag_outlines_Llama2-MedTuned-7b_4bit_all_test_zero_shot_vanilla_rag.pt",
                            "ms-diag_outlines_Llama2-MedTuned-7b_4bit_all_test_zero_shot_instruction_rag.pt",
                            "ms-diag_outlines_Llama2-MedTuned-7b_4bit_all_test_few_shot_vanilla_rag.pt",
                            "ms-diag_outlines_Llama2-MedTuned-7b_4bit_all_test_few_shot_instruction_rag.pt",
                            ], labels="label_encodings_Llama2-MedTuned-7b.pt", load_line=False, load_outlines=True, label_correction=label_correction)])
overall_results["approach"] = ["base"] * 4 + ["rag"] * 4 + ["outlines"] * 4 + ["outlines_rag"] * 4
overall_results.rename(columns={"macro avg.f1-score": "Macro F1-Score",
                                "macro avg.precision": "Macro Precision",
                                "macro avg.recall": "Macro Recall"}, inplace=True)

In [27]:
overall_results.to_csv(paths.THESIS_PATH/"ms_pred_results_prompt7b.csv")

In [25]:
print(generate_latex_table(overall_results, 
                           caption="Macro Precision, Recall and F1-Score for different prompting strategies using Llama2-MedTuned-13B \cite{rohanian2023exploring}",
                           label="tab:ms-diag-13B",
                           metrics=["Macro Precision", "Macro Recall", "Macro F1-Score"]))

\begin{table}[h]
    \centering
    \begin{tabular}{lcccc}
        \toprule
        & \multicolumn{4}{c}{Macro Precision}\\
        \cmidrule(lr){2-5}
        & Base & +S2A & +Outlines & +S2A \& Outlines \\
        \midrule
        zero-shot-vanilla & 0.45 & 0.67 & 0.36 & 0.58 \\
        zero-shot-instruction & 0.13 & 0.26 & 0.70 & 1.00 \\
        few-shot-vanilla & 0.39 & 0.88 & 0.35 & 0.39 \\
        few-shot-instruction & 0.01 & 0.27 & 0.73 & 1.00 \\
        \bottomrule
    \end{tabular}

    \begin{tabular}{lcccc}
        \toprule
        & \multicolumn{4}{c}{Macro Recall}\\
        \cmidrule(lr){2-5}
        & Base & +S2A & +Outlines & +S2A \& Outlines \\
        \midrule
        zero-shot-vanilla & 0.47 & 0.76 & 0.33 & 0.71 \\
        zero-shot-instruction & 0.10 & 0.38 & 0.62 & 1.00 \\
        few-shot-vanilla & 0.25 & 0.98 & 0.33 & 0.50 \\
        few-shot-instruction & 0.17 & 0.46 & 0.84 & 1.00 \\
        \bottomrule
    \end{tabular}

    \begin{tabular}{lcccc}
        \topru

## Valid Label

In [15]:
overall_results[['approach', 'strategy', 'valid_label']]

,approach,strategy,valid_label
0,base,zero_shot_vanilla,0.258621
0,base,zero_shot_instruction,0.172414
0,base,few_shot_vanilla,0.137931
0,base,few_shot_instruction,0.551724
0,rag,zero_shot_vanilla,0.534483
0,rag,zero_shot_instruction,0.551724
0,rag,few_shot_vanilla,0.965517
0,rag,few_shot_instruction,0.982759
0,outlines,zero_shot_vanilla,1.000000
0,outlines,zero_shot_instruction,1.000000


# Comparison with Old Approach (no other class)

In [16]:
results_fso = torch.load(paths.RESULTS_PATH/"ms-diag/ms-diag_outlines_Llama2-MedTuned-13b_4bit_all_test_few_shot_instruction.pt")
other_idx = other_idx = [i for i, label in enumerate(results_fso["labels"]) if label == 3]

results_fso_no = {}
for key in results_fso.keys():
    results_fso_no[key] = [value for i, value in enumerate(results_fso[key]) if i not in other_idx]

torch.save(results_fso_no, paths.RESULTS_PATH/"ms-diag/ms-diag_outlines_Llama2-MedTuned-13b_4bit_all_test_few_shot_instruction_no.pt")

In [17]:
results_fsos2a = torch.load(paths.RESULTS_PATH/"ms-diag/ms-diag_outlines_Llama2-MedTuned-13b_4bit_all_test_few_shot_instruction_rag.pt")
other_idx = other_idx = [i for i, label in enumerate(results_fsos2a["labels"]) if label == 3]

results_fsos2a_no = {}
for key in results_fsos2a.keys():
    results_fsos2a_no[key] = [value for i, value in enumerate(results_fsos2a[key]) if i not in other_idx]

torch.save(results_fsos2a_no, paths.RESULTS_PATH/"ms-diag/ms-diag_outlines_Llama2-MedTuned-13b_4bit_all_test_few_shot_instruction_rag_no.pt")

In [28]:
summary_no = summarize_performance(["ms-diag_outlines_Llama2-MedTuned-13b_4bit_all_test_few_shot_instruction_no.pt",
                                    "ms-diag_outlines_Llama2-MedTuned-13b_4bit_all_test_few_shot_instruction_rag_no.pt"], labels="label_encodings_Llama2-MedTuned-13b.pt", load_line=False, load_outlines=True)
summary_no["strategy"] = ["FSI Outlines", "FSI Outlines + S2A"]

In [29]:
summary_no

,weighted avg.f1-score,weighted avg.support,strategy,valid_label,accuracy,0.precision,0.recall,0.f1-score,0.support,1.precision,1.recall,1.f1-score,1.support,2.precision,2.recall,2.f1-score,2.support,3.precision,3.recall,3.f1-score,3.support,macro avg.precision,macro avg.recall,macro avg.f1-score,macro avg.support,weighted avg.precision,weighted avg.recall
0,0.936780,46.0,FSI Outlines,1.0,0.913043,0.666667,1.0,0.8,4.0,1.000000,0.897436,0.945946,39.0,1.0,1.000000,1.0,3.0,0.0,0.0,0.0,0.0,0.666667,0.724359,0.686486,46.0,0.971014,0.913043
0,0.953924,46.0,FSI Outlines + S2A,1.0,0.934783,1.000000,1.0,1.0,4.0,0.973684,0.948718,0.961039,39.0,1.0,0.666667,0.8,3.0,0.0,0.0,0.0,0.0,0.743421,0.653846,0.690260,46.0,0.977689,0.934783


In [30]:
summary_no.to_csv(paths.THESIS_PATH/"ms_pred_results_prompt13b_no.csv")

In [20]:
for metric in ["precision", "recall", "f1"]:
    cols = [col for col in summary_no.columns if metric in col and not col.startswith("3.")][:3]
    # Row mean
    summary = pd.DataFrame(summary_no[cols].mean(axis=1).round(2), columns = [metric])
    summary["strategy"] = ["FSI Outlines", "FSI Outlines + S2A"]
    display(summary)

,precision,strategy
0,0.89,FSI Outlines
0,0.99,FSI Outlines + S2A


,recall,strategy
0,0.97,FSI Outlines
0,0.87,FSI Outlines + S2A


,f1,strategy
0,0.89,FSI Outlines
0,0.97,FSI Outlines + S2A
